In [ ]:
import pandas as pd
import datetime
import numpy as np
import tweepy
import json
from tweepy import Cursor
import nltk
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize
import re
import string
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk import SnowballStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.util import ngrams
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from bs4 import BeautifulSoup
import random
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from __future__ import print_function
from wordcloud import WordCloud, STOPWORDS
stop = set(stopwords.words('english'))
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
cols = ['sentiment','id','date','query_string','user','text']
df = pd.read_csv('./training_manutd.csv', header= None, names= cols, encoding = "ISO-8859-1")
#df.to_json('./training_manutd.jsonl')

In [ ]:
df['sentiment'] = df['sentiment'].map({0: 0, 4: 1})
df.sentiment.value_counts()
# 1 Posative 0 Negative

In [ ]:
df.text = df.text.str.replace(r'\d+', '')
df.text = df['text'].str.replace('[{}]'.format(string.punctuation), '')
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in (stop)]))

In [ ]:
def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='black',
        stopwords=stop,
        max_words=300,
        max_font_size=40, 
        collocations = False,
        scale=15,
        random_state=42 # chosen at random by flipping a coin; it was heads
    ).generate(str(data))

    fig = plt.figure(1, figsize=(25, 25))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(df['text'])

In [ ]:
word_dist = nltk.FreqDist(df['text'])
word_dist.plot(30)

In [ ]:
pd.set_option('display.max_colwidth', -1)
df.head(2)

In [ ]:
df.drop(['id', 'date', 'query_string', 'user'], axis = 1, inplace= True)
label = pd.factorize(df['sentiment'].values)[0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'],label, train_size=.8,test_size=.2)

In [ ]:
vec = CountVectorizer()
X_train = vec.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(X_train)



In [ ]:
cls = MultinomialNB()
cls.fit(x_train_tfidf , y_train)

In [ ]:
X_test = vec.transform(X_test)
x_test_tfidf = tfidf_transformer.transform(X_test)


In [ ]:
print(cls.predict(x_test_tfidf))
x_test_pred = cls.predict(x_test_tfidf)
x = x_test_pred.reshape(1, -1)
y = y_test.reshape(1, -1)
print(y.size)

In [ ]:
cls.score(x_test_tfidf,y_test)

In [ ]:
import tweepy
import csv
import pandas as pd
####input your credentials here
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

consumer_key = 'RR6miKQ2GKE4cSKj889737iMr'
consumer_secret = 'FLLMME41CmVgCRGrydyoBFR3cmJWTpAmkndyjJI29NvkdYXlrE'
access_token = '2520042060-gTmuDa7yaKLWcJhJLq5igkwckTFBdqCBT9OopIh'
access_token_secret = 'tm6UVzoCPPO7L9Hiq9blymj0pVb5v0YMVEZdl2DLnaVLp'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
#####United Airlines
# Open/Create a file to append data
csvFile = open('TweetsAfter_27_03_2018.csv', 'a', encoding="utf-8")
#Use csv Writer
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search,q="#ManUtd",count=5000,
                           lang="en",since="2017-04-03").items():
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8'),tweet.retweets])
    


In [ ]:
df_manutd = pd.read_csv('./Tweets.csv', header = None, names = ['date_time', 'text'],encoding="ISO-8859-1", sep=',')

In [ ]:
df_manutd.head(1)

In [ ]:
df_manutd['Orig_text'] = df_manutd['text']
df_manutd.text = df_manutd.text.str.replace(r'\d+', '')
df_manutd.text = df_manutd['text'].str.replace('[{}]'.format(string.punctuation), '')
df_manutd['text'] = df_manutd['text'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in (stop)]))
df_manutd['text'] = df_manutd['text'].astype(str)

In [ ]:
df_manutd.text = df_manutd.text.str.replace(r'\d+', '')
df_manutd.text = df_manutd.text.str.replace('http\S+|www.\S+', '', case=False)
df_manutd.text = df_manutd.text.str.replace(r'xe', '', case=False)
df_manutd.text = df_manutd.text.str.replace(r'xf', '', case=False)
df_manutd.text = df_manutd.text.str.replace(r'xb', '', case=False)
df_manutd.text = df_manutd.text.str.replace(r'xa', '', case=False)
df_manutd.text = df_manutd.text.str.replace(r'//', '', case=False)
df_manutd.text = df_manutd.text.str.replace(r'\\', '', case=False)
df_manutd.text = df_manutd.text.str.replace(r'mx', '', case=False)
#m = re.sub(r':.*$', ":", str)
df_manutd.text = df_manutd.text.str.replace(r'brt', '', case=False)
df_manutd.text = df_manutd['text'].str.replace('[{}]'.format(string.punctuation), '')

#stop = set(stopwords.words('english'))
df_manutd['text'] = df_manutd['text'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in (stop)]))

#ttnz = TweetTokenizer()

#f_manutd['tokenized_tweets'] = df_manutd['text'].apply(ttnz.tokenize)

#n_gram = df_manutd['text'].apply(ngrams(nltk.word_tokenize,n))


In [ ]:
df_manutd.head()

In [ ]:
ttnz = TweetTokenizer()

df_manutd['tokenized_tweets'] = df_manutd['text'].apply(ttnz.tokenize)

stemmer = SnowballStemmer(language= 'english', ignore_stopwords= True)
df_manutd['tokenized_tweets'] = df_manutd['tokenized_tweets'].apply(lambda x: [stemmer.stem(y) for y in x])

pd.set_option('display.max_colwidth', -1)
df.head(10)

lemmatizer = WordNetLemmatizer()
df_manutd['tokenized_tweets'] = df_manutd['tokenized_tweets'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])

In [ ]:
txt = df_manutd.text.str.lower().str.replace(r'\|', ' ').str.cat(sep=' ')
top_N = 30
words = nltk.tokenize.word_tokenize(txt)
word_dist = nltk.FreqDist(words)

stopwords = nltk.corpus.stopwords.words('english')
words_except_stop_dist = nltk.FreqDist(w for w in words if w not in stopwords) 

print('All frequencies, including STOPWORDS:')
print('=' * 60)
rslt = pd.DataFrame(word_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])
rslt = pd.DataFrame(words_except_stop_dist.most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word').reset_index()

fig, ax = plt.subplots(figsize=(18,8))
ax = sns.barplot(x='Word',y='Frequency',data=rslt,palette=sns.light_palette("navy",reverse=True,n_colors=21))
plt.xticks(rotation=70)
plt.xticks(size=15)
plt.yticks(size=15)
ax.set(ylim=(0,6000))
ax.set(xlim=(None,20))
ax.set_xlabel('Wrods',size=17)
ax.set_ylabel('Frequency',size=17)

In [ ]:
rslt.head(10)

In [ ]:
from wordcloud import WordCloud, STOPWORDS

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='black',
        stopwords=stop,
        max_words=150,
        max_font_size=60, 
        collocations = False,
        scale=10,
        random_state=42 # chosen at random by flipping a coin; it was heads
    ).generate(str(data))

    fig = plt.figure(1, figsize=(16, 16))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(df_manutd['tokenized_tweets'])
#show_wordcloud(Samsung_Reviews_positive['Reviews'])

In [ ]:
df_manutd_text = vec.transform(df_manutd.text)
df_manutd_text_tfidf = tfidf_transformer.fit_transform(df_manutd_text)
df_pred = cls.predict_proba(df_manutd_text_tfidf)
df_pred_bin = cls.predict(df_manutd_text_tfidf)


In [ ]:
df_pred = pd.DataFrame(df_pred)
df_pred_bin = pd.DataFrame(df_pred_bin)
df_pred_bin = df_pred_bin.reset_index()
df_pred.columns = [['prediction_p', 'prediction_n']]
df_pred = df_pred.reset_index()
df_pred_bin['isPos'] = df_pred_bin[1]

In [ ]:
df_pred_bin.head()

In [ ]:
df_manutd_merge = df_manutd.reset_index()

In [ ]:
df_pred = df_pred_bin.merge(df_pred, left_index=True, right_index=True)

In [ ]:
df_pred = df_pred.merge(df_manutd_merge)

In [ ]:
df_pred.head()

In [ ]:
df_pred.drop(df_pred.columns[[0,1,3]],axis=1,inplace=True)

In [ ]:
df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)

In [ ]:
#df_pred = df_pred.join(df_manutd)

In [ ]:
df_pred = df_pred.drop(['text', 'tokenized_tweets'],axis=1)

In [ ]:
df_pred.head()

In [ ]:
df_pred.date_time = pd.to_datetime(df_pred.date_time)
df_pred.date_time = df_pred.date_time.dt.date
df_pred = df_pred.groupby('date_time').mean().reset_index()

In [ ]:
from googlefinance.client import get_price_data, get_prices_data, get_prices_time_data

params = [
    # Dow Jones
    {
        'q': "MANU",
        'x': "NYSE",
    },
]
period = "1M"
interval = 60*60*24 # 30 minutes
# get open, high, low, close, volume time data (return pandas dataframe)
df_stock = get_prices_time_data(params, period, interval)

In [ ]:
df_stock.index.names = ['date_time']
df_stock.reset_index(inplace=True)

In [ ]:
df_stock.tail()

In [ ]:
import datetime
date = datetime.date(2018, 3, 15)
df_stock = df_stock[df_stock['date_time'] > date]
df_stock.date_time = pd.to_datetime(df_stock.date_time)
df_stock.date_time = df_stock.date_time.dt.date

In [ ]:
df = df_stock.merge(df_pred)

In [ ]:
df.head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots(figsize=(20,10))
t = df['date_time']
s1 = df['MANU_High']
ax1.plot(t, s1, 'b-')
ax1.set_xlabel('Date', size = 15)
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('Stock Price', color='b', size =15)
ax1.tick_params('y', colors='b')

# ax2 = ax1.twinx()
# s2 = df['prediction_p']
# ax2.plot(t, s2, 'r-')
# ax2.set_ylabel('Positve Sentiment', color='r' , size = 15)
# ax2.tick_params('y', colors='r')

ax2 = ax1.twinx()
s2 = df['isPos']
ax2.plot(t, s2, 'g-')
ax2.set_ylabel('Positve Sentiment', color='g' , size = 15)
ax2.tick_params('y', colors='g')

fig.tight_layout()
plt.show()